In [37]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [38]:
!pip -q install mem0ai

In [39]:
from google import genai
from mem0 import Memory

# create client
client = genai.Client()

config = {
    "embedder": {
        "provider": "gemini",
        "config": {
            "model": "models/text-embedding-004",
        }
    },
        "llm": {
        "provider": "gemini",
        "config": {
            "model": "gemini-2.5-flash",
            "temperature": 0.0,
            "max_tokens": 2000,
        }
    },
    "vector_store": {
        "config": {
            "embedding_model_dims": 768,
        }
    }
}

memory = Memory.from_config(config)

In [40]:
memory.add([
    {"role": "user", "content": "Hi, I'm Prajwal Aswar. GenAI Engineer at DataSmith AI."}
], user_id="prajwal")

memory.add([
    {"role": "user", "content": "I love building RAG and AI Agent solutions that actually work in production."}
], user_id="prajwal", metadata={"category": "preferences"})


{'results': [{'id': '112758d5-d048-4c97-9237-97024f7c0acd',
   'memory': 'Loves building RAG and AI Agent solutions that actually work in production',
   'event': 'ADD'}]}

In [41]:
related = memory.search("who am i", user_id="prajwal")
related


{'results': [{'id': '62cc7ed7-7155-4efa-8e3c-7da310c8c1f6',
   'memory': 'Name is Prajwal Aswar',
   'hash': '9b934309a9bb49d903de08c6d1fa9f80',
   'metadata': None,
   'score': 0.385532710288224,
   'created_at': '2025-10-07T02:03:06.306970-07:00',
   'updated_at': None,
   'user_id': 'prajwal'},
  {'id': 'fe5f417b-7fd8-4497-9d40-bce6f4411f79',
   'memory': 'Is a GenAI Engineer at DataSmith AI',
   'hash': 'a268e141b1b8a807f658722efe3bdd29',
   'metadata': None,
   'score': 0.3708313246547329,
   'created_at': '2025-10-07T02:03:06.342443-07:00',
   'updated_at': None,
   'user_id': 'prajwal'},
  {'id': '112758d5-d048-4c97-9237-97024f7c0acd',
   'memory': 'Loves building RAG and AI Agent solutions that actually work in production',
   'hash': 'bd9accc51908a763153e6d9d9991c55e',
   'metadata': {'category': 'preferences'},
   'score': 0.36581861159901086,
   'created_at': '2025-10-07T02:03:12.713504-07:00',
   'updated_at': None,
   'user_id': 'prajwal'}]}

In [42]:
all_memories = memory.get_all(user_id="prajwal")


In [44]:
mem_id = "62cc7ed7-7155-4efa-8e3c-7da310c8c1f6"
wow=memory.get(mem_id)
wow


{'id': '62cc7ed7-7155-4efa-8e3c-7da310c8c1f6',
 'memory': 'Name is Prajwal Aswar',
 'hash': '9b934309a9bb49d903de08c6d1fa9f80',
 'metadata': None,
 'score': None,
 'created_at': '2025-10-07T02:03:06.306970-07:00',
 'updated_at': None,
 'user_id': 'prajwal'}

In [45]:
memory.update(memory_id=mem_id, data="Builds custom Gen AI solutions")
# → {'message': 'Memory updated successfully!'}


{'message': 'Memory updated successfully!'}

In [46]:
history = memory.history(memory_id=mem_id)
history


[{'id': '2e2d0954-7859-4695-8ae1-6b6915f1a8fc',
  'memory_id': '62cc7ed7-7155-4efa-8e3c-7da310c8c1f6',
  'old_memory': None,
  'new_memory': 'Name is Prajwal Aswar',
  'event': 'ADD',
  'created_at': '2025-10-07T02:03:06.306970-07:00',
  'updated_at': None,
  'is_deleted': False,
  'actor_id': None,
  'role': None},
 {'id': 'a181a181-c7ec-4737-996f-f35032273361',
  'memory_id': '62cc7ed7-7155-4efa-8e3c-7da310c8c1f6',
  'old_memory': 'Name is Prajwal Aswar',
  'new_memory': 'Builds custom Gen AI solutions',
  'event': 'UPDATE',
  'created_at': '2025-10-07T02:03:06.306970-07:00',
  'updated_at': '2025-10-07T02:03:43.278939-07:00',
  'is_deleted': False,
  'actor_id': None,
  'role': None}]

In [47]:
[{'event': 'ADD',    'old_memory': None,                     'new_memory': 'Builds custom AI solutions'},
 {'event': 'UPDATE', 'old_memory': 'Builds custom AI solutions',
  'new_memory': 'Builds custom Gen AI solutions'}]


[{'event': 'ADD',
  'old_memory': None,
  'new_memory': 'Builds custom AI solutions'},
 {'event': 'UPDATE',
  'old_memory': 'Builds custom AI solutions',
  'new_memory': 'Builds custom Gen AI solutions'}]

In [48]:
!pip install langchain langgraph

In [49]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

class State(TypedDict):
    """Conversation state passed between nodes"""
    messages: Annotated[list[BaseMessage], add_messages]  # chat history for this request
    mem0_user_id: str                                     # maps to Mem0 user record


In [50]:
# Install the necessary library
!pip install -U langchain-google-genai

In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Google Generative AI chat model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [52]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

def chatbot(state: State):
    global memory  # re‑use the Mem0 instance from Section 1
    msgs = state["messages"]
    uid = state["mem0_user_id"]

    # 1️⃣ Retrieve memories relevant to the latest user msg
    mems = memory.search(msgs[-1].content, user_id=uid)
    print(f"Retrieved Memories: {mems}")

    # Build context string
    if mems["results"]:
        context = "\n".join(f"- {m['memory']}" for m in mems["results"])
    else:
        context = "No relevant information found."

    system = SystemMessage(content=f"""You are a helpful assistant. Use the provided context to personalise your responses.
Relevant information from previous conversations:
{context}""")

    # 2️⃣ Invoke the LLM
    response = llm.invoke([system] + msgs)

    # 3️⃣ Persist the new turn
    memory.add([
        {"role": "user", "content": msgs[-1].content},
        {"role": "assistant", "content": response.content}
    ], user_id=uid)

    return {"messages": [response]}


In [53]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()
print("Graph compiled successfully ✅")


Graph compiled successfully ✅


In [55]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END


def run_conversation(user_input: str, mem0_user_id: str):
    state = {"messages": [HumanMessage(content=user_input)], "mem0_user_id": mem0_user_id}
    result = graph.invoke(state)
    print("🤖", result["messages"][-1].content)

if __name__ == "__main__":
    uid = "customer_prajwal"
    while True:
        inp = input("You: ")
        if inp.lower() in {"quit", "exit", "bye"}:
            break
        run_conversation(inp, uid)

You: what is my name and my year
Retrieved Memories: {'results': [{'id': '7295439b-39c5-4455-a73b-46be63389615', 'memory': 'Is in Last Year', 'hash': '8669dde67e7a1bdcef94df44c0053c5c', 'metadata': None, 'score': 0.5345763289667174, 'created_at': '2025-10-07T02:04:54.769712-07:00', 'updated_at': None, 'user_id': 'customer_prajwal'}, {'id': '5de5b0e3-8304-4862-a9db-13e758fec2d2', 'memory': 'Name is Prajwal', 'hash': '9cecc80191303d638c47633174904c1c', 'metadata': None, 'score': 0.44926736866968026, 'created_at': '2025-10-07T02:04:41.971796-07:00', 'updated_at': None, 'user_id': 'customer_prajwal'}]}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


🤖 Your name is Prajwal and you are in your last year.
You: quit


In [56]:
!pip -q install qdrant_client

In [57]:
collection_name = "mem0_yt"

config = {
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": collection_name,
            "host": "b6c257bf-949d-44d9-8f3b-52cb0eb47727.us-west-2-0.aws.cloud.qdrant.io",
            "port": 6333,
            "api_key": userdata.get("Qdrant_API_KEY")
        }
    },
    "embedder": {
        "provider": "gemini",
        "config": {
            "model": "models/text-embedding-004",
        }
    },
    "llm": { # Explicitly setting the LLM provider
        "provider": "gemini",
        "config": {
            "model": "gemini-2.5-flash",
            "temperature": 0.0,
            "max_tokens": 2000,
        }
    }
}

memory = Memory.from_config(config)

In [58]:
from qdrant_client import QdrantClient
from google.colab import userdata

qdrant = QdrantClient(
    url="https://b6c257bf-949d-44d9-8f3b-52cb0eb47727.us-west-2-0.aws.cloud.qdrant.io:6333",
    api_key=userdata.get("Qdrant_API_KEY") # It's recommended to store your API key securely
)
print(qdrant.get_collections())  # sanity‑check

collections=[CollectionDescription(name='mem0_yt'), CollectionDescription(name='mem0migrations')]


In [59]:
qdrant.create_payload_index(
    collection_name=collection_name,
    field_name="user_id",
    field_schema="keyword"
)


UpdateResult(operation_id=14, status=<UpdateStatus.COMPLETED: 'completed'>)

In [60]:
from qdrant_client.http.models import Distance, VectorParams

try:
    qdrant.delete_collection(collection_name=collection_name)
    print(f"Collection '{collection_name}' deleted successfully.")
except Exception as e:
    print(f"Error deleting collection '{collection_name}': {e}")

qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE) # Set the correct dimension
)
print(f"Collection '{collection_name}' created successfully with dimension 768.")

# Recreate the index on user_id
qdrant.create_payload_index(
    collection_name=collection_name,
    field_name="user_id",
    field_schema="keyword"
)
print(f"Index on 'user_id' created successfully for collection '{collection_name}'.")

Collection 'mem0_yt' deleted successfully.
Collection 'mem0_yt' created successfully with dimension 768.
Index on 'user_id' created successfully for collection 'mem0_yt'.


In [61]:
messages = [
    {"role": "user", "content": "Hi, I'm Prajwal Aswar. GenAI Engineer."},
    {"role": "user", "content": "I love building RAG and AI Agent solutions that work in production."}
]
memory.add(messages, user_id="prajwal")

{'results': [{'id': '9790ff27-4413-41d8-a486-bac0d1adbb29',
   'memory': 'Name is Prajwal Aswar',
   'event': 'ADD'},
  {'id': '267cf1af-7749-440d-a650-85510614c84e',
   'memory': 'Is a GenAI Engineer',
   'event': 'ADD'},
  {'id': '5d4dcbb7-ff30-422e-a36d-6ca60ed739e7',
   'memory': 'Loves building RAG and AI Agent solutions that work in production',
   'event': 'ADD'}]}

In [62]:
from mem0 import MemoryClient

client = MemoryClient(api_key=userdata.get("Mem0_API_KEY"))


In [63]:
messages = [
    {"role": "user",      "content": "Hi, I am Prajwal. GenAI Engineer"},
    {"role": "assistant", "content": "Hi Prajwal"}
]

client.add(messages, user_id="Prajwal_bro")


/usr/local/lib/python3.12/dist-packages/mem0/client/utils.py:38: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


{'results': [{'id': 'f998f37f-62ce-4c57-a611-b224a48a713b',
   'event': 'ADD',
   'memory': 'User Name is Prajwal',
   'structured_attributes': {'day': 7,
    'hour': 9,
    'year': 2025,
    'month': 10,
    'minute': 5,
    'quarter': 4,
    'is_weekend': False,
    'day_of_week': 'tuesday',
    'day_of_year': 280,
    'week_of_year': 41}},
  {'id': '1ee148be-74cf-44c1-af06-39b14b67f4de',
   'event': 'ADD',
   'memory': 'Prajwal Is a GenAI Engineer',
   'structured_attributes': {'day': 7,
    'hour': 9,
    'year': 2025,
    'month': 10,
    'minute': 5,
    'quarter': 4,
    'is_weekend': False,
    'day_of_week': 'tuesday',
    'day_of_year': 280,
    'week_of_year': 41}}]}